<a href="https://colab.research.google.com/github/matpg/RL-Agent-for-Unreal-Engine/blob/main/Codigo_StableBaselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Descarga de Librerias

In [ ]:
!git clone https://github.com/maximecb/gym-minigrid.git
%cd gym-minigrid
!python setup.py install
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines[mpi]==2.10.0
!pip3 install box2d-py

### Importar Librerias

In [3]:
import gym
import gym_minigrid
import numpy as np
from stable_baselines.common.vec_env import *
from stable_baselines.bench.monitor import *
from stable_baselines.common import set_global_seeds
from stable_baselines import PPO2, DDPG, DQN, A2C, ACER, TRPO
from stable_baselines.common.policies import MlpPolicy, CnnPolicy, MlpLstmPolicy, MlpLnLstmPolicy, CnnLstmPolicy, CnnLnLstmPolicy
# from stable_baselines.deepq.policies import MlpPolicy, CnnPolicy
from stable_baselines.common.evaluation import evaluate_policy

### Carga de Entorno Minigrid

In [4]:
# Eenvironement for learning
# Use a separate environement for evaluation
from gym_minigrid.wrappers import *

env_id = 'MiniGrid-LavaCrossingS9N1-v0'
eval_env = gym.make(env_id)
# env = make_vec_env(env, n_envs=2)
eval_env = FlatObsWrapper(eval_env) # Get pixel observations
# env = DirectionObsWrapper(env)
# env = FlatObsWrapper(env)
# env = gym.make('MiniGrid-DoorKey-5x5-v0')
# print(env.observation_space)
# print(env.action_space) # 7

# vectorized environments allow to easily multiprocess training
# we demonstrate its usefulness in the next examples


In [6]:
def make_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.
    
    :param env_id: (str) the environment ID
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = gym.make(env_id)
        env = FlatObsWrapper(env) # Get pixel observations
        # Important: use a different seed for each environment
        env.seed(seed + rank)
        return env
    set_global_seeds(seed)
    return _init

In [7]:
n_process = 8
env = DummyVecEnv([make_env(env_id, i) for i in range(n_process)])
train_env = VecNormalize(env)

In [ ]:
%cd /content/gym-minigrid/

/content/gym-minigrid


In [ ]:
# Use a separate environement for evaluation
# eval_env = gym.make('CartPole-v1')

# Random Agent, before training
# mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

# print(f"mean_reward for random agent:{mean_reward:.2f} +/- {std_reward:.2f}")

### Definicion de Modelo del Agente

In [8]:
agent_model = PPO2(policy='MlpPolicy', env=train_env, n_steps=512, ent_coef=0.0, nminibatches=32, noptepochs=10, verbose=1)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





### Evaluar Agente Ingenuo/Aleatorio

In [9]:
def evaluate(model, eval_env, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    # env = gnwrapper.LoopAnimation(model.get_env()) # Start Xvfb
    env = eval_env
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            # env.render()
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))
        

    # env.display()
    max_episode_reward = np.amax(all_episode_rewards)
    mean_episode_reward = np.mean(all_episode_rewards)
    min_episode_reward = np.amin(all_episode_rewards)
    print(
        "Max reward:", max_episode_reward,
        "Mean reward:", mean_episode_reward,
        "Min reward:", min_episode_reward,
         "Num episodes:", num_episodes)

    return max_episode_reward, mean_episode_reward, min_episode_reward

In [10]:
# Random Agent, before training using property
# from stable_baselines.common.evaluation import evaluate_policy
max_r, mean_r, min_r = evaluate(agent_model, eval_env, num_episodes=100)

# print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Max reward: 0.7611111111111111 Mean reward: 0.007611111111111111 Min reward: 0.0 Num episodes: 100


### Entrenar Agente

In [11]:
agent_model.learn(total_timesteps=2000000, log_interval=10)
train_env.close()

--------------------------------------
| approxkl           | 0.0038542286  |
| clipfrac           | 0.062402345   |
| explained_variance | -0.1          |
| fps                | 907           |
| n_updates          | 1             |
| policy_entropy     | 1.9420274     |
| policy_loss        | -0.0063044047 |
| serial_timesteps   | 512           |
| time_elapsed       | 2.1e-05       |
| total_timesteps    | 4096          |
| value_loss         | 0.20859082    |
--------------------------------------
-------------------------------------
| approxkl           | 0.003936189  |
| clipfrac           | 0.035668947  |
| explained_variance | 0.0654       |
| fps                | 922          |
| n_updates          | 10           |
| policy_entropy     | 1.7578052    |
| policy_loss        | -0.003921165 |
| serial_timesteps   | 5120         |
| time_elapsed       | 37.9         |
| total_timesteps    | 40960        |
| value_loss         | 0.6553676    |
-------------------------------------

### Evaluar Agente Entrenado

In [13]:
# Trained Agent, after training using property
# from stable_baselines.common.evaluation import evaluate_policy
max_r, mean_r, min_r = evaluate(agent_model, eval_env, num_episodes=100)

Max reward: 0 Mean reward: 0.0 Min reward: 0 Num episodes: 100


In [ ]:
import os

# Create save dir
save_dir = "/tmp/gym/"
os.makedirs(save_dir, exist_ok=True)

### Extras

In [ ]:
import gym_minigrid

env = gym.make('MiniGrid-SimpleCrossingEnvUmaze-v0')
for i_episode in range(3):
    obs = env.reset()
    for t_steps in range(100):
        # env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        print(obs)
        if done:
            print("Episode finished after {} timesteps".format(t_steps+1))
            break
env.close()

In [ ]:
!apt update && apt install xvfb
!pip install gym-notebook-wrapper

In [ ]:
# Random Agent, before training using function
mean_reward_before_train = evaluate(agent_model, eval_env, num_episodes=100)

In [ ]:
!pip install tensorboard==2.0.0

In [ ]:
# Trained Agent

mean_reward, std_reward = evaluate_policy(agent_model, train_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
!tensorboard --logdir ./ppo2_minigriddoorkey/ --host localhost 